In [21]:
import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import mnist


In [22]:
with open("generated_data.json", "r") as f:
    data = json.load(f)

In [23]:
X = []
y = []

for item in data:
    keypoints = item["keypoints"][0]
    X.append(np.array(keypoints).flatten())
    y.append(item["label"])
X = np.array(X)

In [24]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

[0 0 0 ... 2 2 2]


In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)
# (X_train, y_train), (X_test, y_test) = mnist.load_data()


In [26]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (32000, 34)
y_train shape: (32000,)
X_test shape: (8000, 34)
y_test shape: (8000,)


In [27]:
num_classes = len(label_encoder.classes_)
input_shape = X_train.shape[1]

In [28]:
X_train = X_train.reshape(-1, 17, 2, 1)
X_test = X_test.reshape(-1, 17, 2, 1)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(label_encoder.classes_)
input_shape = (17, 2, 1)  # 高度、宽度、通道数

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 编译模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/Users/mikeshinoda/Library/Caches/pypoetry/virtualenvs/yolo-pose-estimation-DqWaCygJ-py3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
# model = Sequential()
# model.add(Dense(128, activation="relu", input_shape=(input_shape,)))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation="relu"))
# model.add(Dense(num_classes, activation="softmax"))

# model.compile(
#     optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
# )

In [31]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)


Epoch 1/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 926us/step - accuracy: 0.5578 - loss: 0.8776 - val_accuracy: 0.8728 - val_loss: 0.2769
Epoch 2/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.8620 - loss: 0.2976 - val_accuracy: 0.8903 - val_loss: 0.2361
Epoch 3/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 866us/step - accuracy: 0.8864 - loss: 0.2520 - val_accuracy: 0.9047 - val_loss: 0.2042
Epoch 4/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 867us/step - accuracy: 0.8974 - loss: 0.2279 - val_accuracy: 0.9234 - val_loss: 0.1803
Epoch 5/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - accuracy: 0.9142 - loss: 0.2039 - val_accuracy: 0.9322 - val_loss: 0.1666
Epoch 6/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 956us/step - accuracy: 0.9197 - loss: 0.1950 - val_accuracy: 0.9438 - val_loss: 0.1453
Epoch 7/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 940us/step - accuracy: 0.9267 - loss: 0.1757 - val_accuracy: 0.9497 - val_loss: 0.1338
Epoch 8/50
900/900 ━━━━━━━━━━━━━━━━━━━━ 1s 953us/step - accuracy: 0.9311 - loss: 0.1721 - 

In [32]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("\n测试集准确率:", test_acc)

250/250 - 0s - 350us/step - accuracy: 0.8290 - loss: 0.3590

测试集准确率: 0.8289999961853027


In [33]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print("预测的类别：", y_pred_classes)
print("真实的类别：", y_test)
y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

print(classification_report(y_test_labels, y_pred_labels))


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step
预测的类别： [2 3 3 ... 1 0 2]
真实的类别： [2 1 3 ... 1 0 2]
              precision    recall  f1-score   support

        jump       1.00      1.00      1.00      2047
         run       1.00      0.31      0.47      1967
       stand       1.00      1.00      1.00      2000
        walk       0.59      1.00      0.74      1986

    accuracy                           0.83      8000
   macro avg       0.90      0.83      0.80      8000
weighted avg       0.90      0.83      0.81      8000


In [34]:
model.save("pose.keras")

In [35]:
import pickle

# In[4]:

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

# 保存 LabelEncoder
with open("label_encoder.pkl", "wb") as le_file:
    pickle.dump(label_encoder, le_file)


[0 0 0 ... 2 2 2]


In [36]:
#!/usr/bin/env python
# coding: utf-8

import json
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pickle

# 加载标签编码器
with open("label_encoder.pkl", "rb") as le_file:
    label_encoder = pickle.load(le_file)

# 加载训练好的模型
model = load_model("pose.keras")


# 定义一个函数来预处理关键点数据
def preprocess_keypoints(keypoints):
    """
    预处理关键点数据，使其符合模型输入要求。

    参数:
        keypoints (list): 关键点列表，每个关键点为 [x, y]。

    返回:
        numpy.ndarray: 预处理后的关键点数据，形状为 (17, 2, 1)。
    """
    keypoints_array = np.array(keypoints).flatten()  # 将关键点展平成一维数组
    keypoints_array = keypoints_array.reshape(-1, 17, 2, 1)  # 重新调整形状以匹配模型输入
    return keypoints_array


# 加载 annotations.json 数据
with open("annotations.json", "r") as f:
    annotations = json.load(f)

# 提取关键点并进行预处理
X_new = []
image_paths = []  # 用于存储图像路径（可选）
for item in annotations:
    keypoints = item["keypoints"][0]  # 假设每个条目只有一个关键点集
    X_new.append(preprocess_keypoints(keypoints))
    image_paths.append(item["image_path"])

X_new = np.vstack(X_new)  # 合并所有样本，形状为 (样本数, 17, 2, 1)

# 进行预测
y_pred = model.predict(X_new)
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# 打印预测结果
for img_path, pred_label in zip(image_paths, y_pred_labels):
    print(f"Image: {img_path} --> Predicted Label: {pred_label}")

# 可选：将预测结果保存到一个新的 JSON 文件中
predictions = []
for img_path, pred_label in zip(image_paths, y_pred_labels):
    predictions.append({
        "image_path": img_path,
        "predicted_label": pred_label
    })

with open("predictions.json", "w") as f:
    json.dump(predictions, f, indent=4)

print("预测完成，结果已保存到 predictions.json")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Image: ./g1.png --> Predicted Label: jump
Image: ./g2.png --> Predicted Label: jump
预测完成，结果已保存到 predictions.json
